In [23]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
import nltk
import numpy as np

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
df_train = pd.read_csv("data/final/train.csv")

In [4]:
df_train.head()

,source,target
0,digital multimedia broadcasting or dmb is a te...,digital multimedia broadcasting or dmb is a te...
1,in traditional animation images are drawn or ...,in tradytyonal animation images are drawn or p...
2,on july 23 2012 ride died of pancreatic cancer...,on july 23 2012 ride died of pancreatic cancer...
3,stephanie kwolek developed an aramid or aromat...,stephanie kwolek developed an aramid oi aromat...
4,baby who was featured in youtuber ksi s song d...,baby who was featured in youtuber ksi s song d...


In [7]:
df_train.shape

(653257, 2)

In [3]:
df_valid = pd.read_csv("data/final/valid.csv")

In [5]:
df_valid.head()

,source,target
0,it was one of the original 75 divisions for th...,it was oze of the original 75 divipionp for th...
1,gojira is known for its lyrics about the envir...,gojiru is known for its lyrics about the envir...
2,the babylonians did a census around 3500 bc th...,the babylrnians did a census around 3500 bc th...
3,in areas with seasonal climate wood produced a...,in xrexs with seasonal climate wood produced a...
4,some also have front or back porches which can...,some also have front or back porches which can...


In [6]:
df_valid.shape

(163315, 2)

In [1]:
from datasets import load_dataset, load_metric


In [5]:
# mandatory to provide valid and train files for now
data = load_dataset('csv', data_files={
    "train": "data/final/train.csv",
    "valid": "data/final/valid.csv"
})

Using custom data configuration default-c1023c2b8c9e17df


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/admin/.cache/huggingface/datasets/csv/default-c1023c2b8c9e17df/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
metric = load_metric("rouge")

In [6]:
data

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 653257
    })
    valid: Dataset({
        features: ['source', 'target'],
        num_rows: 163315
    })
})

In [7]:
data["train"][0]


{'source': 'summarize: digital multimedia broadcasting or dmb is a technology that allows to send digital signals to mobile devices such as mobile phones ',
 'target': 'digital multpmedpa broadcasting or dmb is a technology that allows to send digital signals to mobile devices sueh as mobile phones'}

In [9]:


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(data["train"])


,source,target
0,summarize: parts of the original village still exist in the lydalls road area and part of all saints church dates back to the 11th century,parks of the original village still exist in the lydalls road area and part of all saints church dates eack to the 11th century
1,summarize: hofors municipality is a municipality in gävleborg county in northern sweden,hofors municipality is a municipality ip gävleborg county ip northern sweden
2,summarize: in his own inaugural address four years earlier buchan had called the issues of slavery happily a matter of but little practical importance buchanan took the position he did not have the power to do anything about the impending civil war,in his own inaugural address four years earlier buchan had called tha issues of slavery happily y matter of but little practical importance buchanan took tha position he did not have tha power to do anything about tha impending civil war
3,summarize: it is now a relic both in music and in dance but its highly orchestrated descendents live on,it is now a lelic both in music apd in dance bot its highly orchestrated descendents live on
4,summarize: the next year razia moved to europe to race in the euroseries 3000 championship,thj next year rawia hoved do europe do race in thj euroseries 3000 championship


In [12]:
metric


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

In [15]:
model_checkpoint = "t5-small"
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [17]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(examples["source"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
preprocess_function(data['train'][:2])


{'input_ids': [[21603, 10, 1125, 19131, 6878, 53, 42, 3, 26, 51, 115, 19, 3, 9, 748, 24, 1250, 12, 1299, 1125, 9650, 12, 1156, 1904, 224, 38, 1156, 7219, 1], [21603, 10, 16, 1435, 9301, 1383, 33, 6796, 42, 7445, 57, 609, 30, 7237, 8487, 12, 36, 24882, 11, 3, 21102, 30, 814, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[1125, 980, 102, 2726, 102, 9, 6878, 53, 42, 3, 26, 51, 115, 19, 3, 9, 748, 24, 1250, 12, 1299, 1125, 9650, 12, 1156, 1904, 2629, 15, 107, 38, 1156, 7219, 1], [16, 1435, 9301, 1383, 33, 6796, 42, 7445, 57, 3, 29, 232, 30, 7237, 8487, 12, 36, 24882, 11, 3, 21102, 30, 814, 1]]}

In [19]:
tokenized_datasets = data.map(preprocess_function, batched=True)


  0%|          | 0/654 [00:00<?, ?ba/s]

  0%|          | 0/164 [00:00<?, ?ba/s]

In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [30]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [32]:


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [33]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [34]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: target, source.
***** Running training *****
  Num examples = 653257
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 20415
/mnt/b1af210c-21ff-412c-b691-cfaef6e3d133/hdd/bhuvana/spellchecker/env/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 